In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import time
import random
import sys
import pymongo
import urllib.parse
import dns
from mongoengine import *
from mongoengine.context_managers import switch_collection

In [4]:
sleeps = [2,3,4]
def otp():
    client = pymongo.MongoClient('mongodb+srv://bilalm:' + urllib.parse.quote_plus('Codemarket.123') + '@codemarket-staging.k16z7.mongodb.net/dreamjobpal?retryWrites=true&w=majority')
    my_db = client['dreamjobpal']
    time.sleep(random.choice(sleeps))
    print("Linkedin sent you an OTP to your email.")
    otp_db = my_db.linkedin_otp
    otp_db.insert({"linkedin_login_url": Email_id, "Status":"OTP sent"})
# execute "python otp.py <email> <otp>" in another terminal
    flag = True
    while flag:
        data = otp_db.find({"linkedin_login_url": Email_id, "Status":"OTP updated"})
        data = list(data)
        # print(data)
        # print(len(data))
        if len(data) == 1:
            # print(data[0]['OTP'])
            otp = data[0]['OTP']
            # print("otp is: ",otp)
            otp_db.update_many( 
            {"linkedin_login_url":Email_id, "Status":"OTP updated"}, 
            { "$set":{ "Status":"Login complete" }},)
            break
        time.sleep(5)
    # otp = int(input("Please enter the OTP recieved at registered mobile number: "))
    # driver.current_url
    submit_otp = driver.find_element_by_name("pin")
    submit_otp.send_keys(otp)
    submit_otp.send_keys(Keys.RETURN)
        
def chat_scroll():
    chats = chat_list.find_elements_by_xpath("//div[@class='msg-conversation-listitem__link msg-overlay-list-bubble__convo-item   msg-overlay-list-bubble__convo-item--v3']")
    chat_len = len(chats)
    for l in chats :
        driver.execute_script("arguments[0].scrollIntoView();", l)
        time.sleep(1)
        chat_check = chat_list.find_elements_by_xpath("//div[@class='msg-conversation-listitem__link msg-overlay-list-bubble__convo-item   msg-overlay-list-bubble__convo-item--v3']")
#         print(len(chat_check))
    if len(chat_check) > chat_len:
        chat_scroll()
    return chat_check

PATH = r"C:\Users\BILAL\Projects\LinkedInScraper\chromedriver.exe"
# PATH= r"/usr/local/bin/chromedriver"
chrome_options = Options()
driver = webdriver.Chrome(PATH,options=chrome_options)


driver.get("https://www.linkedin.com/login")
driver.find_element_by_id("username").send_keys('donnybegins@gmail.com')
password = driver.find_element_by_id("password")
time.sleep(random.choice(sleeps))
password.send_keys('myaccount')
password.send_keys(Keys.RETURN)

if 'https://www.linkedin.com/checkpoint' in driver.current_url:
        otp()
    print('Login...')
    
my_db.linkedin_otp.drop()



        
        
        

In [34]:
chat_list = driver.find_element_by_xpath("//div[@class='msg-conversations-container__conversations-list msg-overlay-list-bubble__conversations-list']")
# chat_list

chat_container = chat_scroll()



In [51]:
soup1 = BeautifulSoup(driver.page_source, 'html.parser')


In [52]:
chats = soup1.find_all('div', attrs={"class": "msg-conversation-listitem__link msg-overlay-list-bubble__convo-item msg-overlay-list-bubble__convo-item--v3"})


In [53]:
len(chats)

40

In [55]:
for enu,chat in enumerate(chats):
    chat = chats[enu].find('mark', attrs={"class":"msg-conversation-card__unread-count"})
    if chat:
        print("New Message")
        chat_container[enu].send_keys(Keys.RETURN)
        time.sleep(random.choice(sleeps))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        chat_box = soup.find('div', {"class":"msg-overlay-conversation-bubble"})
        
        conversation_list = chat_box.find('ul', {"class":"msg-s-message-list-content list-style-none full-width"})
        conversation = conversation_list.find_all('li')
        #if len(conversation) < 4:
        container = chat_box.find('h4')
        name = container.find('a').text.strip()
        print(name)
        first_name = name.split(' ')[0]
#             print(first_name)
        last_name = name.split( ' ')[1]
#             print(last_name)
        time.sleep(random.choice(sleeps))

        link = conversation[0].find_all('a')
        link = link[1]['href']

        profile_link= "https://www.linkedin.com"+link
        print(profile_link)
            
        my_link = conversation[2].find('a')['href']
        print(my_link)
#         link=conversation[0].find('a')['href']
#         profile_link= "https://www.linkedin.com"+link
#         print(profile_link)
        
        conversation.reverse()
        for num,messages in enumerate(conversation):
            link_check = messages.find('a', attrs={"data-control-name":"view_profile"})
            if link_check is None:
                continue
            print("LINKKKKKKKKK",link_check['href'])
            if link_check['href'] == my_link:
                print("LINK CHECK",link_check['href'])
                conversation_msg = conversation[num].p.text.strip()
                print(conversation_msg)
            else:
                continue
            if conversation_msg == 'I have a great job opportunity for you' or conversation_msg == 'I have a great job opportunity for you.':
                print("Replying")

                message = f"www.soozzi.com/job?linkedinurl=={profile_link}&&firstname={first_name}&&lastname={last_name}"
        #             message = "hii"
                reply = driver.find_element_by_xpath("//div[@aria-label='Write a message…']")
                time.sleep(random.choice(sleeps))
                reply.send_keys(message)
                send_reply = driver.find_element_by_xpath("//button[@type='submit']")
                time.sleep(random.choice(sleeps))

                send_reply.send_keys(Keys.RETURN)

                close_chat = driver.find_element_by_xpath("//button[@data-control-name='overlay.close_conversation_window']")
                close_chat.send_keys(Keys.RETURN)
                print("Replied")
                time.sleep(random.choice(sleeps))
                break
            else: 
                close_chat = driver.find_element_by_xpath("//button[@data-control-name='overlay.close_conversation_window']")
                time.sleep(random.choice(sleeps))
                close_chat.send_keys(Keys.RETURN)
                time.sleep(random.choice(sleeps))
            print("complete1")
#                 break
#             else:
#                 continue
    print("complete2")
    #break            



        

New Message
Bilal Momin
https://www.linkedin.com/in/bilal-momin-189218192/
/in/donny-koay-417462202/
LINKKKKKKKKK /in/bilal-momin-189218192/
LINKKKKKKKKK /in/donny-koay-417462202/
LINK CHECK /in/donny-koay-417462202/
adasadsxassa
complete1
LINKKKKKKKKK /in/donny-koay-417462202/
LINK CHECK /in/donny-koay-417462202/
www.soozzi.com/job?linkedinurl==https://www.linkedin.com/in/bilal-momin-189218192/&&firstname=Bilal&&lastname=Momin


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-control-name='overlay.close_conversation_window']"}
  (Session info: chrome=87.0.4280.141)
